Scoring/Conceding in next 10 actions model - Model Tuning and Building - GBM

In [1]:
import pandas as pd
pd.options.display.max_columns = None

from expected_vaep_model.config import exp_vaep_chain_output_path, player_stats_file_path, exp_vaep_player_stats_output_path
from expected_vaep_model.models.predict_player_stats import predict_player_stats

import warnings
warnings.filterwarnings("ignore")    

In [2]:
exp_vaep_chain_output_path

'/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_match_chains_v5.csv'

In [3]:
player_stats_file_path

'/Users/ciaran/Documents/Projects/AFL/data/player_stats.csv'

In [4]:
chains = pd.read_csv(exp_vaep_chain_output_path)
player_stats = pd.read_csv(player_stats_file_path)

In [5]:
player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season
28950,2023F4_Collingwood_BrisbaneLions,Collingwood,Patrick Lipinski,2023F4,Patrick_Lipinski,MEDIUM_FORWARD,CD_I1003130,25,190,86,1,RIGHT,VIC,2016.0,2017.0,Eltham (Vic)/Northern U18/Western Bulldogs,28.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-07-17,58,0,0.0,NaN,0,3,NaN,NaN,NaN,0,NaN,8,NaN,NaN,NaN,NaN,81.3,16,59,NaN,NaN,NaN,1,0,0.0,0,0,NaN,12,0,NaN,NaN,NaN,2,NaN,1,NaN,NaN,NaN,4,NaN,2,0,NaN,134.0,0,NaN,NaN,0,NaN,4,NaN,1,NaN,3,5,1,3,16,2,8,NaN,NaN,Substitute,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023.0
28951,2023F4_Collingwood_BrisbaneLions,Collingwood,Scott Pendlebury,2023F4,Scott_Pendlebury,MIDFIELDER,CD_I260257,35,191,86,10,LEFT,VIC,2005.0,2006.0,Sale (Vic)/Gippsland U18,5.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1988-01-07,77,0,0.0,NaN,4,3,NaN,NaN,NaN,0,NaN,9,NaN,NaN,NaN,NaN,75.0,24,102,NaN,NaN,NaN,0,1,100.0,0,1,NaN,8,0,NaN,NaN,NaN,4,NaN,1,NaN,NaN,NaN,16,NaN,5,1,NaN,324.0,2,NaN,NaN,1,NaN,7,NaN,1,NaN,2,4,0,6,22,2,13,NaN,NaN,Centre,FINAL_TEAM,Centre,Midfield,NaN,NaN,2023.0
28952,2023F4_Collingwood_BrisbaneLions,Collingwood,Steele Sidebottom,2023F4,Steele_Sidebottom,MIDFIELDER,CD_I280965,32,184,86,22,RIGHT,VIC,2008.0,2009.0,Congupna (Vic)/Murray U18,11.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1991-01-02,86,0,1.0,NaN,1,3,NaN,NaN,NaN,0,NaN,6,NaN,NaN,NaN,NaN,75.0,20,98,NaN,NaN,NaN,1,0,50.0,0,1,NaN,8,0,NaN,NaN,NaN,4,NaN,3,NaN,NaN,NaN,12,NaN,5,0,NaN,403.0,1,NaN,NaN,1,NaN,7,NaN,2,NaN,2,7,0,3,20,3,14,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,NaN,NaN,2023.0
28953,2023F4_Collingwood_BrisbaneLions,Collingwood,Tom Mitchell,2023F4,Tom_Mitchell,MIDFIELDER,CD_I291856,30,182,88,6,LEFT,WA,2011.0,2013.0,Ashburton (Vic)/Claremont (WAFL)/Sydney/Hawthorn,21.0,fatherSon,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-05-31,72,0,0.0,NaN,1,2,NaN,NaN,NaN,0,NaN,10,NaN,NaN,NaN,NaN,87.5,24,124,NaN,NaN,NaN,0,1,0.0,2,0,NaN,13,0,NaN,NaN,NaN,3,NaN,3,NaN,NaN,NaN,11,NaN,4,0,NaN,246.0,1,NaN,NaN,1,NaN,7,NaN,0,NaN,6,13,1,7,25,2,15,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023.0
28954,2023F4_Collingwood_BrisbaneLions,Collingwood,Will Hoskin-Elliott,2023F4,Will_Hoskin-Elliott,MIDFIELDER_FORWARD,CD_I291720,30,186,82,32,RIGHT,VIC,2011.0,2012.0,North Sunshine (Vic)/Western U18/GWS,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-09-02,89,0,0.0,NaN,0,2,NaN,NaN,NaN,0,NaN,7,NaN,NaN,NaN,NaN,50.0,12,58,NaN,NaN,NaN,0,0,0.0,1,0,NaN,1,0,NaN,NaN,NaN,3,NaN,5,NaN,NaN,NaN,11,NaN,5,0,NaN,175.0,4,NaN,NaN,1,NaN,5,NaN,0,NaN,0,2,0,0,13,1,6,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023.0


In [6]:
def get_player_totals_by_match(chains):
    return chains.groupby(['Match_ID', 'Player', 'Team'])[['xScore', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']].sum().reset_index().sort_values(by=['Match_ID', "Player", "Team"])

player_value = get_player_totals_by_match(chains)
player_stats = player_stats.merge(player_value, how = "left", on = ['Match_ID', 'Player', 'Team'])

def get_receiver_totals_by_match(chains):
    return chains.groupby(['Match_ID', 'Receiver'])[['exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']].sum().reset_index().sort_values(by=['Match_ID', "Receiver"])

chains['Receiver'] = chains['Player'].shift(-1)
receiver_value = get_receiver_totals_by_match(chains)
receiver_value = receiver_value.rename(columns = {
    "Receiver":"Player",
    "exp_vaep_value":"exp_vaep_value_received"})
player_stats = player_stats.merge(receiver_value[['Match_ID', 'Player', 'exp_vaep_value_received']], how = "left", on = ['Match_ID', 'Player'])


In [7]:
player_value.tail()

,Match_ID,Player,Team,xScore,exp_vaep_value,exp_offensive_value,exp_defensive_value
28498,2023F4_Collingwood_BrisbaneLions,Scott Pendlebury,Collingwood,4.018784,12.288110,14.292848,-2.004738
28499,2023F4_Collingwood_BrisbaneLions,Steele Sidebottom,Collingwood,4.186815,5.518497,9.461985,-3.943488
28500,2023F4_Collingwood_BrisbaneLions,Tom Mitchell,Collingwood,0.000000,8.772737,9.195726,-0.422988
28501,2023F4_Collingwood_BrisbaneLions,Will Hoskin-Elliott,Collingwood,0.000000,2.934727,2.915613,0.019114
28502,2023F4_Collingwood_BrisbaneLions,Zac Bailey,Brisbane Lions,7.094413,8.710697,10.142595,-1.431898


In [8]:
player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,exp_vaep_value,exp_offensive_value,exp_defensive_value,exp_vaep_value_received
28950,2023F4_Collingwood_BrisbaneLions,Collingwood,Patrick Lipinski,2023F4,Patrick_Lipinski,MEDIUM_FORWARD,CD_I1003130,25,190,86,1,RIGHT,VIC,2016.0,2017.0,Eltham (Vic)/Northern U18/Western Bulldogs,28.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-07-17,58,0,0.0,NaN,0,3,NaN,NaN,NaN,0,NaN,8,NaN,NaN,NaN,NaN,81.3,16,59,NaN,NaN,NaN,1,0,0.0,0,0,NaN,12,0,NaN,NaN,NaN,2,NaN,1,NaN,NaN,NaN,4,NaN,2,0,NaN,134.0,0,NaN,NaN,0,NaN,4,NaN,1,NaN,3,5,1,3,16,2,8,NaN,NaN,Substitute,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023.0,1.708957,2.448745,3.000071,-0.551326,2.366989
28951,2023F4_Collingwood_BrisbaneLions,Collingwood,Scott Pendlebury,2023F4,Scott_Pendlebury,MIDFIELDER,CD_I260257,35,191,86,10,LEFT,VIC,2005.0,2006.0,Sale (Vic)/Gippsland U18,5.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1988-01-07,77,0,0.0,NaN,4,3,NaN,NaN,NaN,0,NaN,9,NaN,NaN,NaN,NaN,75.0,24,102,NaN,NaN,NaN,0,1,100.0,0,1,NaN,8,0,NaN,NaN,NaN,4,NaN,1,NaN,NaN,NaN,16,NaN,5,1,NaN,324.0,2,NaN,NaN,1,NaN,7,NaN,1,NaN,2,4,0,6,22,2,13,NaN,NaN,Centre,FINAL_TEAM,Centre,Midfield,NaN,NaN,2023.0,4.018784,12.288110,14.292848,-2.004738,14.121951
28952,2023F4_Collingwood_BrisbaneLions,Collingwood,Steele Sidebottom,2023F4,Steele_Sidebottom,MIDFIELDER,CD_I280965,32,184,86,22,RIGHT,VIC,2008.0,2009.0,Congupna (Vic)/Murray U18,11.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1991-01-02,86,0,1.0,NaN,1,3,NaN,NaN,NaN,0,NaN,6,NaN,NaN,NaN,NaN,75.0,20,98,NaN,NaN,NaN,1,0,50.0,0,1,NaN,8,0,NaN,NaN,NaN,4,NaN,3,NaN,NaN,NaN,12,NaN,5,0,NaN,403.0,1,NaN,NaN,1,NaN,7,NaN,2,NaN,2,7,0,3,20,3,14,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,NaN,NaN,2023.0,4.186815,5.518497,9.461985,-3.943488,4.647661
28953,2023F4_Collingwood_BrisbaneLions,Collingwood,Tom Mitchell,2023F4,Tom_Mitchell,MIDFIELDER,CD_I291856,30,182,88,6,LEFT,WA,2011.0,2013.0,Ashburton (Vic)/Claremont (WAFL)/Sydney/Hawthorn,21.0,fatherSon,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-05-31,72,0,0.0,NaN,1,2,NaN,NaN,NaN,0,NaN,10,NaN,NaN,NaN,NaN,87.5,24,124,NaN,NaN,NaN,0,1,0.0,2,0,NaN,13,0,NaN,NaN,NaN,3,NaN,3,NaN,NaN,NaN,11,NaN,4,0,NaN,246.0,1,NaN,NaN,1,NaN,7,NaN,0,NaN,6,13,1,7,25,2,15,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023.0,0.000000,8.772737,9.195726,-0.422988,12.657220
28954,2023F4_Collingwood_BrisbaneLions,Collingwood,Will Hoskin-Elliott,2023F4,Will_Hoskin-Elliott,MIDFIELDER_FORWARD,CD_I291720,30,186,82,32,RIGHT,VIC,2011.0,2012.0,North Sunshine (Vic)/Western U18/GWS,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-09-02,89,0,0.0,Na

In [9]:
exp_vaep_player_stats_output_path

'/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_player_stats_v5.csv'

In [ ]:
player_stats.to_csv(exp_vaep_player_stats_output_path, index=False)

In [10]:
player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,exp_vaep_value,exp_offensive_value,exp_defensive_value,exp_vaep_value_received
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,NaN,0.000000,4.841868,2.757104,2.084763,1.668863
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,NaN,0.000000,6.470962,4.936769,1.534193,5.893544
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,NaN,0.000000,6.335013,7.174054,-0.839041,11.085720
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig (WA)/Subiaco (WAFL),7.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1990-06-07,84,0,0.0,0.0,0,4,1.0,50.0,2.0,0,12.5,2,0.0,0.0,0.0,5.0,60.9,23,95,14.0,12.0,0.0,0,0,0.0,0,0,2.0,2,0,0.0,0.0,0.0,2,3.0,6,57.1,7.0,7.0,21,10.5,8,0,0.0,641.0,4,8.0,6.3,11,0.0,1,0.0,0,4.0,0,1,0,0,16,6,14,NaN,NaN,Back Pocket Left,FINAL_TEAM,Back-Pocket,Back,2021.0,0.0,NaN,0.000000,-4.729125,1.558721,-6.287847,12.016121
4,202101_BrisbaneLions_Sydney,Brisbane Lions,Dayne Zorko,202101,Dayne_Zorko,MIDFIELDER,CD_I261224,32,175,78,15,RIGHT,QLD,2011.0,2012.0,Surfers Paradise (Qld)/Broadbeach (Qld),NaN,other,https://s.afl.com.au/staticfile/AFL Tenant

In [11]:
predict_player_stats(scored_chains = exp_vaep_chain_output_path, 
                     player_stats_file_path = "/Users/ciaran/Documents/Projects/AFL/data/player_stats.csv", 
                     scored_player_stats_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_player_stats_v5.csv")

In [14]:
scored_data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_player_stats_v5.csv")
scored_data.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,exp_vaep_value,exp_offensive_value,exp_defensive_value,exp_vaep_value_received
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,NaN,0.000000,4.841868,2.757104,2.084763,1.668863
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,NaN,0.000000,6.470962,4.936769,1.534193,5.893544
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,NaN,0.000000,6.335013,7.174054,-0.839041,11.085720
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig (WA)/Subiaco (WAFL),7.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1990-06-07,84,0,0.0,0.0,0,4,1.0,50.0,2.0,0,12.5,2,0.0,0.0,0.0,5.0,60.9,23,95,14.0,12.0,0.0,0,0,0.0,0,0,2.0,2,0,0.0,0.0,0.0,2,3.0,6,57.1,7.0,7.0,21,10.5,8,0,0.0,641.0,4,8.0,6.3,11,0.0,1,0.0,0,4.0,0,1,0,0,16,6,14,NaN,NaN,Back Pocket Left,FINAL_TEAM,Back-Pocket,Back,2021.0,0.0,NaN,0.000000,-4.729125,1.558721,-6.287847,12.016121
4,202101_BrisbaneLions_Sydney,Brisbane Lions,Dayne Zorko,202101,Dayne_Zorko,MIDFIELDER,CD_I261224,32,175,78,15,RIGHT,QLD,2011.0,2012.0,Surfers Paradise (Qld)/Broadbeach (Qld),NaN,other,https://s.afl.com.au/staticfile/AFL Tenant

In [15]:
scored_data.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,exp_vaep_value,exp_offensive_value,exp_defensive_value,exp_vaep_value_received
28950,2023F4_Collingwood_BrisbaneLions,Collingwood,Patrick Lipinski,2023F4,Patrick_Lipinski,MEDIUM_FORWARD,CD_I1003130,25,190,86,1,RIGHT,VIC,2016.0,2017.0,Eltham (Vic)/Northern U18/Western Bulldogs,28.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-07-17,58,0,0.0,NaN,0,3,NaN,NaN,NaN,0,NaN,8,NaN,NaN,NaN,NaN,81.3,16,59,NaN,NaN,NaN,1,0,0.0,0,0,NaN,12,0,NaN,NaN,NaN,2,NaN,1,NaN,NaN,NaN,4,NaN,2,0,NaN,134.0,0,NaN,NaN,0,NaN,4,NaN,1,NaN,3,5,1,3,16,2,8,NaN,NaN,Substitute,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023.0,1.708957,2.448745,3.000071,-0.551326,2.366989
28951,2023F4_Collingwood_BrisbaneLions,Collingwood,Scott Pendlebury,2023F4,Scott_Pendlebury,MIDFIELDER,CD_I260257,35,191,86,10,LEFT,VIC,2005.0,2006.0,Sale (Vic)/Gippsland U18,5.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1988-01-07,77,0,0.0,NaN,4,3,NaN,NaN,NaN,0,NaN,9,NaN,NaN,NaN,NaN,75.0,24,102,NaN,NaN,NaN,0,1,100.0,0,1,NaN,8,0,NaN,NaN,NaN,4,NaN,1,NaN,NaN,NaN,16,NaN,5,1,NaN,324.0,2,NaN,NaN,1,NaN,7,NaN,1,NaN,2,4,0,6,22,2,13,NaN,NaN,Centre,FINAL_TEAM,Centre,Midfield,NaN,NaN,2023.0,4.018784,12.288110,14.292848,-2.004738,14.121951
28952,2023F4_Collingwood_BrisbaneLions,Collingwood,Steele Sidebottom,2023F4,Steele_Sidebottom,MIDFIELDER,CD_I280965,32,184,86,22,RIGHT,VIC,2008.0,2009.0,Congupna (Vic)/Murray U18,11.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1991-01-02,86,0,1.0,NaN,1,3,NaN,NaN,NaN,0,NaN,6,NaN,NaN,NaN,NaN,75.0,20,98,NaN,NaN,NaN,1,0,50.0,0,1,NaN,8,0,NaN,NaN,NaN,4,NaN,3,NaN,NaN,NaN,12,NaN,5,0,NaN,403.0,1,NaN,NaN,1,NaN,7,NaN,2,NaN,2,7,0,3,20,3,14,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,NaN,NaN,2023.0,4.186815,5.518497,9.461985,-3.943488,4.647661
28953,2023F4_Collingwood_BrisbaneLions,Collingwood,Tom Mitchell,2023F4,Tom_Mitchell,MIDFIELDER,CD_I291856,30,182,88,6,LEFT,WA,2011.0,2013.0,Ashburton (Vic)/Claremont (WAFL)/Sydney/Hawthorn,21.0,fatherSon,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-05-31,72,0,0.0,NaN,1,2,NaN,NaN,NaN,0,NaN,10,NaN,NaN,NaN,NaN,87.5,24,124,NaN,NaN,NaN,0,1,0.0,2,0,NaN,13,0,NaN,NaN,NaN,3,NaN,3,NaN,NaN,NaN,11,NaN,4,0,NaN,246.0,1,NaN,NaN,1,NaN,7,NaN,0,NaN,6,13,1,7,25,2,15,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023.0,0.000000,8.772737,9.195726,-0.422988,12.657220
28954,2023F4_Collingwood_BrisbaneLions,Collingwood,Will Hoskin-Elliott,2023F4,Will_Hoskin-Elliott,MIDFIELDER_FORWARD,CD_I291720,30,186,82,32,RIGHT,VIC,2011.0,2012.0,North Sunshine (Vic)/Western U18/GWS,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-09-02,89,0,0.0,Na